# 탐색
## 5-1 미니맥스법을 활용한 틱택토
### 5-1-1 미니맥스법이란?
미니맥스법(Minimax algorithm)은 플레이어는 자신에게 있어서 최선의 수를 선택하고, 상대는 최악의 수를 선택한다는 가정에서 가장 좋은 수를 찾는 탐색 알고리즘이다. <br>
두 사람이 대결하는 유한(게임에서 둘 수 있는 숫자가 유한) 확정(무작위적인 요소가 없음) 완전 정보(모든 정보가 두 플레이어 모두에게 공개된 상태) 게임에서는 미니맥스법을 많이 사용한다. <br>
<br>
리프 노드의 상태 평가로부터 각 노드의 상태 평가를 다음 규칙에 따라 계산한다.
- 자신의 국면(사각) 노드는 그 자식 노드 상태 평가의 최댓값을 상태 가치로 함
- 상대방의 국면(원) 노드는 그 자식 노드 상태 평가의 최소값을 상태 가치로 함

### 5-1-2 틱택토 작성
먼저, 틱택토 게임 국면을 나타내는 클래스 `State`를 작성한다.
- State의 멤버 변수

pieces : 내 돌의 배치 || enemy_pieces : 상대방 돌의 배치 <br>
둘은 3$\times$3의 눈금을 길이 9의 배열로 표현한다. 돌이 존재하는 경우는 1, 존재하지 않는 경우는 0이 된다.

- State 메소드

`next()`는 행동에 따라 다음 상태를 취득한다. 행동은 돌을 배치하는 매스의 위치를 0~8의 숫자로 지정한다. <br>
`legal_actions()`는 둘 수 있는 수의 리스트로 선택 가능한 행동을 의미한다. 비어 있는 모든 위치에 해당한다.

In [1]:
# 틱택토 생성
import random

# 게임 상태
class State:
    # 초기화
    def __init__(self, pieces=None, enemy_pieces=None):
        # 돌 배치
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9
        
    # 돌의 수 취득
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        return count
    
    # 패배 여부 판정
    def is_lose(self):
        # 돌 3개 연결 여부
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x< 0 or 2 < x or \
                self.enemy_pieces[x + y * 3] == 0:
                    return False
                
            x = x + dx
            y = y + dy
        return True
    
        # 패배 여부 판정
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False
        
    # 무승부 여부 판정
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9
    
    # 게임 종료 여부 판정
    def is_done(self):
        return self.is_lose() or self.is_draw()
    
    # 다음 상태 얻기
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)
    
    # 둘 수 있는 수의 리스트 얻기
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions
    
    # 선 수 여부 판정
    def is_first_player(self):
        return self.pieces_count(self.piecces) == self.piece_count(self.enemy_pieces)
    
    # 문자열 표시
    def __str__(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        str = ''
        for i in range(9):
            if self.pieces[i] == 1:
                str += ox[0]
            elif self.enemy_pieces[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
        return str

### 5-1-3 랜덤 행동 선택
무작위로 행동을 선택하는 함수를 작성한다. 

In [2]:
# 랜덤 행동 선택
def random_actions(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions) - 1)]

### 5-1-4 랜덤 대 랜덤의 대전

In [4]:
# 랜덤과 랜덤의 대전

# 상태 생성
state = State()

# 게임 종료 시까지 반복
while True:
    # 게임 종료 시
    if state.is_done():
        break
        
    # 행동 얻기
    action = random_action(state)
    
    # 다음 상태 얻기
    state = state.next(action)
    
    # 문자열 표시
    print(state)
    print()

### 5-1-5 미니맥스법을 활용한 사애 가치 계산
미니맥스법으로 상태의 가치를 계산하는 함수를 작성한다. `State`를 전달하면 상태가치(승리 확률이 클수록 높음)를 반환한다.
- 게임이 종료된 경우

상태가 게임 종료인 경우에는 상태 가치 '-1: 패배', '0: 무승부'를 반환한다.

- 게임이 종료되지 않은 경우

상태가 게임이 종료되지 않으면 둘 수 있는 수별로 상태 가치를 계산하고, 그 최댓값을 반환한다. 둘 수 있는 수의 상태 가치는 재귀적으로 게임이 종료될 때까지 조사해 계산한다. <br>


In [6]:
# 미니맥스법을 활용한 상태 가치 계산
def mini_max(state):
    # 패배 시 상태 가치 -1
    if state.is_lose():
        return -1
    
    # 무승부 시 상태 가치 0
    if state.is_draw():
        return 0
    
    # 둘 수 있는 수의 상태 가치 계산
    best_score = -float('inf')
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_score = score
            
    # 둘 수 있는 수의 상태 가치값 중 최대값 선택
    return best_score

In [7]:
# 미니맥스법을 활용한 행동 선택
def mini_max_action(state):
    # 둘 수 있는 수의 상태 가치 계산
    best_action = 0
    best_score = -float('inf')
    str = ['','']
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_action = action
            best_score = score
            
        str[0] = '{}{:2d},'.format(str[0], action)
        str[1] = '{}{:2d},'.format(str[1], score)
    print('action:', str[0], '\nscore: ', str[1], '\n')
    
    # 둘 수 있는 수의 상태 가치 최댓값을 가진 행동 반환
    return best_action

In [8]:
# 미니맥스법과 랜덤의 대전

# 상태 생성
state = State()

# 게임 종료 시까지 반복
while True:
    # 게임 종료 시
    if state.is_done():
        break
        
    # 행동 얻기
    if state.is_first_player():
        action = mini_max_action(state)
    else:
        action = random_action(state)
        
    # 다음 상태 얻기
    state = state.next(action)
    
    # 문자열 표시
    print(state)
    print()